# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
output_weather_file = "../WeatherPy/output_data/city_weather.csv"
weather_data = pd.read_csv(output_weather_file)

In [8]:
# Store latitude and longitude in locations
locations = weather_data[["lat", "long"]]

# Fill NaN values and convert to float
humidity = weather_data["humidity"].astype(float)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [10]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)

# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [11]:
# Plot Heatmap
#fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# Narrow down the DataFrame to find your ideal weather condition. 
# Drop a row by condition  
# A max temperature >75F and <=80F lower than 80 degrees but higher than 75.
# very few clouds (<5%) cloudiness.
# Wind speed < 10 (light breeze)


myCities = weather_data.loc[weather_data["temp"]>75]
myCities = myCities.loc[myCities["temp"]<80]
myCities = myCities.loc[myCities["wind speed"]<10]
myCities = myCities.loc[myCities["clouds"]<5]

#myCities.head()
#weather_data.max()

In [13]:
#myCities.count()
myCities

,city,lat,long,temp,max_temp,humidity,wind speed,clouds
144,Altamira,-3.20,-52.21,77.49,77.49,69,1.88,3
215,Dayong,25.02,118.29,75.20,75.20,60,4.47,0
230,Havelock,34.88,-76.90,75.06,77.00,57,8.05,1
241,Mapiripan,2.89,-72.13,78.57,78.57,69,2.17,0
280,Paducah,37.08,-88.60,75.29,78.80,73,6.93,1
286,Atambua,-9.11,124.89,77.68,77.68,74,5.19,0
405,Gilbues,-9.83,-45.34,78.94,78.94,34,5.01,1
429,Dekernes,31.09,31.59,77.00,77.00,73,5.82,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = myCities
hotel_df

,city,lat,long,temp,max_temp,humidity,wind speed,clouds
144,Altamira,-3.20,-52.21,77.49,77.49,69,1.88,3
215,Dayong,25.02,118.29,75.20,75.20,60,4.47,0
230,Havelock,34.88,-76.90,75.06,77.00,57,8.05,1
241,Mapiripan,2.89,-72.13,78.57,78.57,69,2.17,0
280,Paducah,37.08,-88.60,75.29,78.80,73,6.93,1
286,Atambua,-9.11,124.89,77.68,77.68,74,5.19,0
405,Gilbues,-9.83,-45.34,78.94,78.94,34,5.01,1
429,Dekernes,31.09,31.59,77.00,77.00,73,5.82,0


In [15]:
hotel_df["hotel"]=" "

In [16]:
hotel_df["city"]

144     Altamira
215       Dayong
230     Havelock
241    Mapiripan
280      Paducah
286      Atambua
405      Gilbues
429     Dekernes
Name: city, dtype: object

In [17]:
hotel_df.reset_index()

,index,city,lat,long,temp,max_temp,humidity,wind speed,clouds,hotel
0,144,Altamira,-3.20,-52.21,77.49,77.49,69,1.88,3,
1,215,Dayong,25.02,118.29,75.20,75.20,60,4.47,0,
2,230,Havelock,34.88,-76.90,75.06,77.00,57,8.05,1,
3,241,Mapiripan,2.89,-72.13,78.57,78.57,69,2.17,0,
4,280,Paducah,37.08,-88.60,75.29,78.80,73,6.93,1,
5,286,Atambua,-9.11,124.89,77.68,77.68,74,5.19,0,
6,405,Gilbues,-9.83,-45.34,78.94,78.94,34,5.01,1,
7,429,Dekernes,31.09,31.59,77.00,77.00,73,5.82,0,


In [26]:
# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "39.952583,-75.16522",  # philadelphia coords,
    "rankby": "distance",
    "type": "hotel",
    "key": gkey,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    params["location"] = f"{hotel_df.loc[index, 'lat']},{hotel_df.loc[index, 'long']}"

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        #print(results)
        hotel_df.loc[index, 'hotel'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

------------
------------
------------
------------
------------
------------
------------
------------


In [24]:
hotel_df

,city,lat,long,temp,max_temp,humidity,wind speed,clouds,hotel
144,Altamira,-3.20,-52.21,77.49,77.49,69,1.88,3,Macario Artefatos de Concreto
215,Dayong,25.02,118.29,75.20,75.20,60,4.47,0,Yuantou Avenue
230,Havelock,34.88,-76.90,75.06,77.00,57,8.05,1,Havelock-Craven County Library
241,Mapiripan,2.89,-72.13,78.57,78.57,69,2.17,0,cyberplay2
280,Paducah,37.08,-88.60,75.29,78.80,73,6.93,1,Geotech Engineering
286,Atambua,-9.11,124.89,77.68,77.68,74,5.19,0,Pondok Pesantren Al Muhajirin Atambua
405,Gilbues,-9.83,-45.34,78.94,78.94,34,5.01,1,Banco do Brasil
429,Dekernes,31.09,31.59,77.00,77.00,73,5.82,0,Ahmed younis لتجارة السيارات


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map